In [35]:
import pandas as pd

import numpy as np

import csv,sqlite3

In [2]:
conn = sqlite3.connect("predictions_sql.db")
cursor = conn.cursor()

In [17]:
df = pd.read_csv("predictions.csv", header=None, names=["user_id", "item_id", "rating_5"])

In [21]:
df.to_sql("predictions_tb", conn, index=False)

In [4]:
# read from sql db
# pred_df = pd.read_sql_query("select * from predictions_tb limit 5;", conn)

# pred_df

,user_id,item_id,rating_5
0,A2DJV5P2NTWEK0,0001981692,4.191430
1,A2DJV5P2NTWEK0,0002008211,4.699676
2,A2DJV5P2NTWEK0,0002254565,4.702403
3,A2DJV5P2NTWEK0,0002552493,4.645525
4,A2DJV5P2NTWEK0,0002710188,4.791867


In [55]:
# store pickle files
metadata = pd.read_pickle("refined_metadata.pkl")

metadata1 = metadata[['asin', 'title', 'authors', 'lexile']]

In [56]:
cursor.execute('''DROP TABLE IF EXISTS metadata_tbl''')
conn.commit()

metadata1.to_sql("metadata_tbl", conn, index=False)

In [57]:
# store utility file
utility = pd.read_pickle("utility_df.pkl")

In [60]:
utility.shape

(3822, 6052)

In [59]:
cursor.execute('''DROP TABLE IF EXISTS utility_tbl''')
conn.commit()

utility.to_sql("utility_tbl", conn, index=False)

OperationalError: too many columns on utility_tbl